In [ ]:
import sys
sys.path.append('../')

import pandas as pd

from dynalign.experiments.defaults import DATASETS, ALL_ALIGNERS, POSTHOC_ALIGNERS
from dynalign.experiments.parsers import MetricsParser

In [ ]:
mp = MetricsParser(aligners=ALL_ALIGNERS)
df = pd.DataFrame(mp.parse())

In [ ]:
sub_df = df[(df.metric == "calculation_time")].drop(["epoch", "metric"], axis=1)


with pd.option_context("display.max_rows", 1000):
    times_df = (
        sub_df[sub_df.aligner != "Node2Vec"]
        .groupby(["aligner", "dataset", "snapshot"])
        .agg("mean")
        .groupby(["aligner", "dataset"])
        .agg("sum")
        .drop("run", axis=1)
    )
    display(times_df)

In [ ]:
node2vec_times = (
    sub_df[sub_df.aligner == "Node2Vec"]
    .drop(["aligner"], axis=1)
    .groupby(["dataset", "snapshot"])
    .agg("mean")
    .drop("run", axis=1)
    .reset_index()

)
node2vec_times = (
    node2vec_times[node2vec_times.snapshot != 0]
    .groupby("dataset")
    .agg("sum")
    .drop("snapshot", axis=1)
)
node2vec_times

In [ ]:
times_df = times_df.reset_index().pivot(
    columns=["dataset"], index=["aligner"], values="value"
)

for aligner in POSTHOC_ALIGNERS:
    times_df.loc[aligner] = times_df.loc[aligner] + node2vec_times.value
    
times_df["total"] = times_df.sum(axis=1)
times_df.sort_values(by="total")